In [3]:
import socket
from PIL import Image
import time
import numpy as np
import time
import matplotlib.pyplot as plt
from pynq import Overlay, allocate

overlay = Overlay('/home/xilinx/jupyter_notebooks/all4_flex_pipe.bit')
dma = overlay.axi_dma_0

# socket TCP/IP
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('192.168.2.99', 916))
print("Waiting for connection...")
s.listen(1)

conn, addr = s.accept()
print('Connected to', addr)

data_in=conn.recv(2)

data_size_in = int.from_bytes(data_in, byteorder='big')
print(data_size_in)

data_in=conn.recv(2)

data_size_out0 = int.from_bytes(data_in, byteorder='big')
print(data_size_out0)

data_in=conn.recv(2)

data_size_out1 = int.from_bytes(data_in, byteorder='big')
print(data_size_out1)

in_buffer = allocate(shape=data_size_in, dtype=np.uint8)

while True:
    # receive from PC
    data_in = conn.recv(data_size_in)
    
    # Check the most significant bit of the data_in
    if data_in[3] & 0x80 == 0:
        out_buffer = allocate(shape=data_size_out0, dtype=np.uint8)
    else:
        out_buffer = allocate(shape=data_size_out1, dtype=np.uint8)

    # send to FPGA
    in_buffer[:] = np.frombuffer(data_in, dtype=np.uint8)[:]
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # send to PC
    data_out = out_buffer[:].tobytes()
    conn.sendall(data_out)
    
    # Free the buffer
    out_buffer.freebuffer()

Waiting for connection...
Connected to ('192.168.2.1', 51242)
260
256
260


IndexError: index out of range